In [2]:
import pandas as pd

movies = pd.read_csv('final_data.csv')

In [4]:
import json

mean_std_json = open('cleaned_data/mean_std_2019.json', 'r')

mean_std = json.load(mean_std_json)

In [5]:
mean_std

{'prod_company|vote_average': [0.4487883695853029, 10.668354897368522],
 'prod_company|vote_count': [20740.32898799314, 106699.12125248263],
 'prod_company|profit': [1428029630.034949, 9511154233.807024],
 'prod_company|rev_budget_ratio': [1984.1456002432185, 94191.57350114622],
 'writer|vote_average': [-0.06938705723231973, 6.475768683117582],
 'writer|vote_count': [11080.568162282963, 26244.503498267368],
 'writer|profit': [875935954.0886366, 2717950033.0740314],
 'writer|rev_budget_ratio': [1682.2327030091246, 83991.91105626336],
 'director|vote_average': [0.06802063404051828, 5.770254937359396],
 'director|vote_count': [9466.29387113672, 20567.215022609063],
 'director|profit': [762719283.4817607, 2211250563.383148],
 'director|rev_budget_ratio': [1695.6738540337337, 68691.93197750473],
 'cast|vote_average': [0.4935719715215528, 16.938832677947325],
 'cast|vote_count': [35009.501064352386, 80819.37673916346],
 'cast|profit': [2643438879.8517766, 6530267994.184139],
 'cast|rev_budge

In [6]:
name_basics = pd.read_csv('cleaned_data/relevant_name_basics.txt', delimiter='|')

In [7]:
name_basics

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous",tt0072308
1,nm0000004,John Belushi,1949,1982,"actor,writer,soundtrack","tt0080455, tt0078723"
2,nm0000006,Ingrid Bergman,1915,1982,"actress,soundtrack,producer","tt0038109, tt0038787"
3,nm0000007,Humphrey Bogart,1899,1957,"actor,soundtrack,producer","tt0033870, tt0034583"
4,nm0000008,Marlon Brando,1924,2004,"actor,soundtrack,director","tt0068646, tt0044081, tt0078788"
...,...,...,...,...,...,...
397100,nm9993605,Kirk 'Ish' Ishitani,\N,\N,producer,tt0859635
397101,nm9993606,Big John Love,\N,\N,producer,tt0859635
397102,nm9993607,Joel Seligstein,\N,\N,producer,tt0859635
397103,nm9993608,Noah Shah,\N,\N,producer,tt0859635


In [33]:
num_genres = 4
num_prod_companies = 5
num_directors = 3
num_writers = 5
num_others = 10
num_cast = 10

In [37]:
def get_tconst(name):
    try:
        return name_basics[name_basics['primaryName'] == name][['nconst']].values[0][0]
    except:
        return ''
    
def set_tconsts(staff_list):
    for i in range(len(staff_list)):
        staff_list[i] = get_tconst(staff_list[i])
        
    return staff_list

def fill_missing(feature_list, max_number):
    i = len(feature_list)
    
    while i < max_number:
        feature_list.append('')
        i += 1
        
    return feature_list

In [49]:
get_tconst('Fred Astaire')

'nm0634240'

In [25]:
get_tconst('John Belushi')

'nm0000004'

In [26]:
set_tconsts(['Fred Astaire', 'John Belushi'])

['nm0000001', 'nm0000004']

In [27]:
set_tconsts(['Spaghetti Monster', 'John Belushi'])

['', 'nm0000004']

In [38]:
fill_missing(['Fred Astaire', 'John Belushi'], 5)

['Fred Astaire', 'John Belushi', '', '', '']

In [40]:
nontrivial_col_types = [['prod_company_1',
  'prod_company_2',
  'prod_company_3',
  'prod_company_4',
  'prod_company_5'],
 ['writer_1', 'writer_2', 'writer_3', 'writer_4', 'writer_5'],
 ['director_1', 'director_2', 'director_3'],
 ['cast_1',
  'cast_2',
  'cast_3',
  'cast_4',
  'cast_5',
  'cast_6',
  'cast_7',
  'cast_8',
  'cast_9',
  'cast_10'],
 ['others_1',
  'others_2',
  'others_3',
  'others_4',
  'others_5',
  'others_6',
  'others_7',
  'others_8',
  'others_9',
  'others_10']]

In [45]:
def assign_features(cur_dict, col, col_values, num_cols):
    for i in range(num_cols):
        cur_dict[col + '_' + str(i)] = col_values[i]
    return cur_dict

In [46]:
assign_features({}, 'cast', fill_missing(['Fred Astaire', 'John Belushi'], num_cast), num_cast)

{'cast_0': 'Fred Astaire',
 'cast_1': 'John Belushi',
 'cast_2': '',
 'cast_3': '',
 'cast_4': '',
 'cast_5': '',
 'cast_6': '',
 'cast_7': '',
 'cast_8': '',
 'cast_9': ''}

In [57]:
def get_df_without_scores(budget, runtime, genres, prod_companies, writers, directors, actors, others):
    """
    prod_companies, writers, directors, actors, others will be a list of names which I'll then join on their id's
    which I'll then use to compute the score
    """
    genres = fill_missing(genres, num_genres)
    prod_companies = fill_missing(prod_companies, num_prod_companies)
    writers = fill_missing(set_tconsts(writers), num_writers)
    directors = fill_missing(set_tconsts(directors), num_directors)
    actors = fill_missing(set_tconsts(actors), num_cast)
    others = fill_missing(set_tconsts(others), num_others)
        
    df_dict = {}
    
    df_dict['budget'] = budget
    df_dict['runtime'] = runtime
    
    df_dict = assign_features(df_dict, 'genre', genres, num_genres)
    df_dict = assign_features(df_dict, 'prod_company', prod_companies, num_prod_companies)
    df_dict = assign_features(df_dict, 'writer', writers, num_writers)
    df_dict = assign_features(df_dict, 'director', directors, num_directors)
    df_dict = assign_features(df_dict, 'cast', actors, num_cast)
    df_dict = assign_features(df_dict, 'others', others, num_others)

#     print(df_dict)
    return pd.DataFrame([df_dict])

In [59]:
pd.set_option(("display.max_columns"), 50)

In [60]:
get_df_without_scores(2, 4, ['Action', 'Comedy'], ['20th Century Fox'], ['Christopher Nolan'],
                      ['Christopher Nolan'], ['John Belushi'], ['Fred Astaire'])

,budget,runtime,genre_0,genre_1,genre_2,genre_3,prod_company_0,prod_company_1,prod_company_2,prod_company_3,prod_company_4,writer_0,writer_1,writer_2,writer_3,writer_4,director_0,director_1,director_2,cast_0,cast_1,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7,cast_8,cast_9,others_0,others_1,others_2,others_3,others_4,others_5,others_6,others_7,others_8,others_9
0,2,4,Action,Comedy,,,20th Century Fox,,,,,nm0634240,,,,,nm0634240,,,nm0000004,,,,,,,,,,nm0000001,,,,,,,,,
